In [1]:
from os.path import join as pathjoin
import sys
sys.path.insert(0, '/home/mlepekhin/Non-thematic-Text-Classification/code/allennlp_experiments')
from data_processing import *
from interpretation import *
from models import *
from training import *

Let's download all the data.

In [2]:
DATA_DIR = '/home/mlepekhin/data'
MODELS_DIR = '/home/mlepekhin/models'
MODEL_ID = 'allennlp_xlm_roberta_topic_ru_small_prefix50'
CHECKPOINTS_DIR = pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints')
BEST_MODEL = pathjoin(CHECKPOINTS_DIR, 'best.th')

In [3]:
transformer_model = 'xlm-roberta-base'
MAX_TOKENS = 512
all_filename = 'ru_small_all_50.csv'

In [4]:
all_df = pd.read_csv(f'/home/mlepekhin/ru-gpts/generated_texts/{all_filename}')
all_df = all_df.sample(frac=1, random_state=42)
all_df.target = all_df.topic
all_df.iloc[:all_df.shape[0] // 2, :].to_csv(f'train_{all_filename}')
all_df.iloc[all_df.shape[0] // 2:, :].to_csv(f'test_{all_filename}')
all_df.head()

,Unnamed: 0,Unnamed: 0.1,prefix,target,text,topic
521,21,1,"если вы сделали заказ с получением в сц , мпв ...",crime,"если вы сделали заказ с получением в сц, мпв и...",crime
737,37,7,портрет у входа на виадук : чтобы добраться до...,education,портрет у входа на виадук : чтобы добраться до...,education
740,40,0,игра deepolis с лёгкостью выдерживает статус м...,games,игра deepolis с лёгкостью выдерживает статус м...,games
660,60,0,"обновлен раздел "" фотографии "" - добавлено нес...",music,"обновлен раздел "" фотографии "" - добавлено нес...",music
411,11,1,"в номинации "" электронная коммерция в рунете ""...",business,"в номинации "" электронная коммерция в рунете ""...",business


In [5]:
all_topics = np.unique(pd.read_csv(pathjoin(DATA_DIR, 'awd_ru_topics100.csv')).topic.values)
#all_labels = np.unique(pd.read_csv('/home/mlepekhin/data/smart_genre/ru/all.csv').target.values)
print(all_topics)

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54. 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67. 68. 69. 70. 71.
 72. 73. 74. 75. 76. 77. 78. 79. 80. 81. 82. 83. 84. 85. 86. 87. 88. 89.
 90. 91. 92. 93. 94. 95. 96. 97. 98. 99.]


In [6]:
#pd.read_csv('/home/mlepekhin/data/smart_genre/ru/all.csv').dropna().sample(random_state=42, frac=0.1).to_csv('smart_genre_sample.csv')

In [7]:
train_dataset_reader = build_transformer_dataset_reader(transformer_model, lower=True)
val_dataset_reader = build_transformer_dataset_reader(transformer_model, lower=True)

train_data, dev_data = read_data(
    f'train_{all_filename}', 
    f'test_{all_filename}',
    train_dataset_reader, 
    val_dataset_reader
)

vocab = build_vocab(train_data + dev_data)

train_data.index_with(vocab)
dev_data.index_with(vocab)

01272021 10:51:47|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-config.json from cache at /home/mlepekhin/.cache/torch/transformers/762ddd751172e9d3229e5da17a459eee6c0dfdc237c718944d0b1a85f06c7e1e.2b0f807393c56e8861a31cd67d2fc0b45d71d9735dd47dd66afb650f90b6d2a8
01272021 10:51:47|INFO|transformers.configuration_utils| Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 250002
}

01272021 10:51:48|INFO|transformers.tokenization_

01272021 10:51:58|INFO|transformers.tokenization_utils| loading file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-sentencepiece.bpe.model from cache at /home/mlepekhin/.cache/torch/transformers/0c370616ddfc06067c0634160f749c2cf9d8da2c50e03a2617ce5841c8df3b1d.309f0c29486cffc28e1e40a2ab0ac8f500c203fe080b95f820aa9cb58e5b84ed
01272021 10:51:59|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-config.json from cache at /home/mlepekhin/.cache/torch/transformers/762ddd751172e9d3229e5da17a459eee6c0dfdc237c718944d0b1a85f06c7e1e.2b0f807393c56e8861a31cd67d2fc0b45d71d9735dd47dd66afb650f90b6d2a8
01272021 10:51:59|INFO|transformers.configuration_utils| Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
 

Reading data
<class 'data_processing.ClassificationDatasetReader'> train_ru_small_all_50.csv


01272021 10:52:21|INFO|allennlp.data.vocabulary| Fitting token dictionary from dataset.



Building the vocabulary


In [8]:
model = build_pool_transformer_model(vocab, transformer_model)
if torch.cuda.is_available():
    cuda_device = 2
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
print(cuda_device)

Building the model


01272021 10:52:21|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-config.json from cache at /home/mlepekhin/.cache/torch/transformers/762ddd751172e9d3229e5da17a459eee6c0dfdc237c718944d0b1a85f06c7e1e.2b0f807393c56e8861a31cd67d2fc0b45d71d9735dd47dd66afb650f90b6d2a8
01272021 10:52:21|INFO|transformers.configuration_utils| Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 250002
}

01272021 10:52:21|INFO|transformers.modeling_util

2


In [9]:
!rm -rf {CHECKPOINTS_DIR}
!mkdir -p {CHECKPOINTS_DIR}

In [10]:
train_loader, dev_loader = build_data_loaders(train_data, dev_data)

# You obviously won't want to create a temporary file for your training
# results, but for execution in binder for this course, we need to do this.

trainer = build_classifier_trainer(
    model,
    pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints'),
    train_loader,
    dev_loader,
    6,
    cuda_device=cuda_device
)
print("Starting training")
trainer.train()
print("Finished training")

01272021 10:52:40|INFO|allennlp.training.optimizers| Number of trainable parameters: 278051338
01272021 10:52:40|WARNING|allennlp.training.trainer| You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
01272021 10:52:40|INFO|allennlp.training.trainer| Beginning training.
01272021 10:52:40|INFO|allennlp.training.trainer| Epoch 0/5
01272021 10:52:40|INFO|allennlp.training.trainer| Worker 0 memory usage MB: 5160.94
01272021 10:52:41|INFO|allennlp.training.trainer| GPU 0 memory usage MB: 11
01272021 10:52:41|INFO|allennlp.training.trainer| GPU 1 memory usage MB: 7193
01272021 10:52:41|INFO|allennlp.training.trainer| GPU 2 memory usage MB: 1990
01272021 10:52:41|INFO|allennlp.training.trainer| GPU 3 memory usage MB: 18470
01272021 10:52:41|INFO|allennlp.training.trainer| Training


Starting training


01272021 10:52:58|INFO|allennlp.training.trainer| Validating


01272021 10:53:04|INFO|allennlp.training.tensorboard_writer|                        Training |  Validation
01272021 10:53:04|INFO|allennlp.training.tensorboard_writer| accuracy           |     0.216  |     0.392
01272021 10:53:04|INFO|allennlp.training.tensorboard_writer| gpu_0_memory_MB    |    11.000  |       N/A
01272021 10:53:04|INFO|allennlp.training.tensorboard_writer| gpu_1_memory_MB    |  7193.000  |       N/A
01272021 10:53:04|INFO|allennlp.training.tensorboard_writer| gpu_2_memory_MB    |  1990.000  |       N/A
01272021 10:53:04|INFO|allennlp.training.tensorboard_writer| gpu_3_memory_MB    |  18470.000  |       N/A
01272021 10:53:04|INFO|allennlp.training.tensorboard_writer| loss               |     2.238  |     1.847
01272021 10:53:04|INFO|allennlp.training.tensorboard_writer| reg_loss           |     0.000  |     0.000
01272021 10:53:04|INFO|allennlp.training.tensorboard_writer| worker_0_memory_MB |  5160.940  |       N/A


01272021 10:53:08|INFO|allennlp.training.checkpointer| Best validation performance so far. Copying weights to '/home/mlepekhin/models/allennlp_xlm_roberta_topic_ru_small_prefix50/checkpoints/best.th'.
01272021 10:53:09|INFO|allennlp.training.trainer| Epoch duration: 0:00:28.246111
01272021 10:53:09|INFO|allennlp.training.trainer| Estimated training time remaining: 0:02:21
01272021 10:53:09|INFO|allennlp.training.trainer| Epoch 1/5
01272021 10:53:09|INFO|allennlp.training.trainer| Worker 0 memory usage MB: 5160.94
01272021 10:53:09|INFO|allennlp.training.trainer| GPU 0 memory usage MB: 11
01272021 10:53:09|INFO|allennlp.training.trainer| GPU 1 memory usage MB: 7193
01272021 10:53:09|INFO|allennlp.training.trainer| GPU 2 memory usage MB: 18470
01272021 10:53:09|INFO|allennlp.training.trainer| GPU 3 memory usage MB: 18470
01272021 10:53:09|INFO|allennlp.training.trainer| Training


...

01272021 10:55:48|INFO|allennlp.training.checkpointer| Best validation performance so far. Copying weights to '/home/mlepekhin/models/allennlp_xlm_roberta_topic_ru_small_prefix50/checkpoints/best.th'.
01272021 10:56:00|INFO|allennlp.training.trainer| Epoch duration: 0:00:39.442788
01272021 10:56:00|INFO|allennlp.training.trainer| Estimated training time remaining: 0:00:40
01272021 10:56:00|INFO|allennlp.training.trainer| Epoch 5/5
01272021 10:56:00|INFO|allennlp.training.trainer| Worker 0 memory usage MB: 5160.94
01272021 10:56:01|INFO|allennlp.training.trainer| GPU 0 memory usage MB: 11
01272021 10:56:01|INFO|allennlp.training.trainer| GPU 1 memory usage MB: 7193
01272021 10:56:01|INFO|allennlp.training.trainer| GPU 2 memory usage MB: 18470
01272021 10:56:01|INFO|allennlp.training.trainer| GPU 3 memory usage MB: 24079
01272021 10:56:01|INFO|allennlp.training.trainer| Training


01272021 10:56:18|INFO|allennlp.training.trainer| Validating


01272021 10:56:24|INFO|allennlp.training.tensorboard_writer|                        Training |  Validation
01272021 10:56:24|INFO|allennlp.training.tensorboard_writer| accuracy           |     0.988  |     0.968
01272021 10:56:24|INFO|allennlp.training.tensorboard_writer| gpu_0_memory_MB    |    11.000  |       N/A
01272021 10:56:24|INFO|allennlp.training.tensorboard_writer| gpu_1_memory_MB    |  7193.000  |       N/A
01272021 10:56:24|INFO|allennlp.training.tensorboard_writer| gpu_2_memory_MB    |  18470.000  |       N/A
01272021 10:56:24|INFO|allennlp.training.tensorboard_writer| gpu_3_memory_MB    |  24079.000  |       N/A
01272021 10:56:24|INFO|allennlp.training.tensorboard_writer| loss               |     0.062  |     0.165
01272021 10:56:24|INFO|allennlp.training.tensorboard_writer| reg_loss           |     0.000  |     0.000
01272021 10:56:24|INFO|allennlp.training.tensorboard_writer| worker_0_memory_MB |  5160.940  |       N/A


01272021 10:56:27|INFO|allennlp.training.checkpointer| Best validation performance so far. Copying weights to '/home/mlepekhin/models/allennlp_xlm_roberta_topic_ru_small_prefix50/checkpoints/best.th'.
01272021 10:56:33|INFO|allennlp.training.trainer| Epoch duration: 0:00:32.898927
01272021 10:56:33|INFO|allennlp.training.checkpointer| loading best weights


Finished training


In [11]:
!rm "{CHECKPOINTS_DIR}"/*.json "{CHECKPOINTS_DIR}"/model* "{CHECKPOINTS_DIR}"/training*
model.vocab.save_to_files(pathjoin(MODELS_DIR, MODEL_ID, 'vocab'))
!ls -lh "{CHECKPOINTS_DIR}"

01272021 10:56:35|INFO|filelock| Lock 140374294696000 acquired on /home/mlepekhin/models/allennlp_xlm_roberta_topic_ru_small_prefix50/vocab/.lock
01272021 10:56:36|INFO|filelock| Lock 140374294696000 released on /home/mlepekhin/models/allennlp_xlm_roberta_topic_ru_small_prefix50/vocab/.lock


total 1,1G
-rw-rw-r-- 1 mlepekhin mlepekhin 1,1G янв 27 10:56 best.th
drwxrwxr-x 4 mlepekhin mlepekhin 4,0K янв 27 10:52 log
